<a href="https://colab.research.google.com/github/bbenip/tetris-ai/blob/main/data/modify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pre-processing the data to get it into a Numpy array

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/bbenip/tetris-ai.git
!cd tetris-ai/data

Mounted at /content/drive
Cloning into 'tetris-ai'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 121 (delta 10), reused 9 (delta 2), pack-reused 93
Receiving objects: 100% (121/121), 36.39 MiB | 34.73 MiB/s, done.
Resolving deltas: 100% (42/42), done.


Install dependencies

In [ ]:
# @title Decompress game files
games_password = '' # @param {type:"string"}

!pip install unrar
!unrar x -p{games_password} /content/tetris-ai/data/games/games.rar /content/tetris-ai/data/games

Streaming output truncated to the last 5000 lines.
Extracting  /content/tetris-ai/data/games/5389.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5388.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5387.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5386.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5385.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5384.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5383.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5382.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5381.json                       38%  OK 
Extracting  /content/tetris-ai/data/games/5380.json                       38%  OK 
Extracting  /content/tetris-a

In [ ]:
!pip install h5py

In [ ]:
import os
import json
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
MOVES_FOLDER_PATH = os.path.dirname('/content/tetris-ai/data/games/')
print(MOVES_FOLDER_PATH)

/content/tetris-ai/data/games


In [ ]:
MOVES_TABLE = {
    "MOVE_LEFT": 0,
    "MOVE_RIGHT": 1,
    "ROTATE_LEFT": 2,
    "ROTATE_RIGHT": 3,
    "ROTATE_180": 4,
    "SOFT_DROP": 5,
    "HARD_DROP": 6,
    "DAS_LEFT": 7,
    "DAS_RIGHT": 8,
    "HOLD_BLOCK": 9
}

In [ ]:
# Load moves from JSON file
def loadMoves(filepath):
    heldBlocks = np.empty((0, 1), dtype=int)
    activeBlocks = np.empty((0, 1), dtype=int)
    blockPositions = np.empty((0, 2), dtype=int)
    blockRotations = np.empty((0, 1), dtype=int)
    movesGrids = np.empty((0, 1), dtype=object)
    movesY = np.empty((0, 1), dtype=int)
    with open(filepath, 'rb') as f:
        data = json.load(f)
        for move in data:
            heldBlock = move["heldBlock"]["id"] + 1 if move["heldBlock"] else 0 #Shift id values up by 1 to allow null
            activeBlock = move["activeBlock"]["id"] + 1 #Just so the block ids match up
            blockPos = [move["activeBlock"]["pos"]["x"], move["activeBlock"]["pos"]["y"]]
            blockRot = move["activeBlock"]["rot"]
            currGrid = np.array(move["oldBoard"]) #(20,10)
            nextMove = MOVES_TABLE[move["move"]]

            # Append to the NumPy array
            heldBlocks = np.append(heldBlocks, heldBlock)
            #print(heldBlocks.shape)
            activeBlocks = np.append(activeBlocks, activeBlock)
            blockPositions = np.vstack((blockPositions, blockPos))
            blockRotations = np.append(blockRotations, blockRot)
            movesGrids = np.append(movesGrids, currGrid)
            movesY = np.append(movesY, nextMove)
    movesGrids = movesGrids.reshape(-1, 20, 10)
    return (heldBlocks, activeBlocks, blockPositions, blockRotations, movesGrids, movesY)

In [ ]:
import h5py
DATA_FOLDER_PATH = "/content/tetris-ai/data/"
TRAIN_DATASET_FILE_NAME = "train_data.hdf5"
TEST_DATASET_FILE_NAME = "test_data.hdf5"
TRAIN_DATASET_FILE_PATH = os.path.join(DATA_FOLDER_PATH, TRAIN_DATASET_FILE_NAME)
TEST_DATASET_FILE_PATH = os.path.join(DATA_FOLDER_PATH, TEST_DATASET_FILE_NAME)

In [ ]:
def createDatasets(filepath, heldBlocks, activeBlocks, blockPositions, blockRotations, movesGrids, movesY):
  with h5py.File(filepath, "w") as f:
      dsetHeldBlocks = f.create_dataset("X_held_block", data=heldBlocks, maxshape=(None,))
      dsetActiveBlock = f.create_dataset("X_active_block", data=activeBlocks, maxshape=(None,))
      dsetBlockPositions = f.create_dataset("X_block_pos", data=blockPositions, maxshape=(None, 2))
      dsetBlockRotations = f.create_dataset("X_block_rot", data=blockRotations, maxshape=(None,))
      dsetGrids = f.create_dataset("X_grid", data=movesGrids, maxshape=(None, 20, 10))

      dsetY = f.create_dataset("Y", data=movesY, maxshape=(None,))

In [ ]:
def appendToDataset(filepath, datasetName, arr):
  with h5py.File(filepath, "a") as hf:
    hf[datasetName].resize((hf[datasetName].shape[0] + arr.shape[0]), axis = 0)
    hf[datasetName][-arr.shape[0]:] = arr

def appendAllToDataset(filepath, heldBlocks, activeBlocks, blockPositions, blockRotations, movesGrids, movesY):
  appendToDataset(filepath, 'X_held_block', heldBlocks)
  appendToDataset(filepath, 'X_active_block', activeBlocks)
  appendToDataset(filepath, 'X_block_pos', blockPositions)
  appendToDataset(filepath, 'X_block_rot', blockRotations)
  appendToDataset(filepath, 'X_grid', movesGrids)

  appendToDataset(filepath, 'Y', movesY)

In [ ]:
def printHDF5File(filepath):
  with h5py.File(filepath, 'r') as f1:
      for dset in f1.keys():
        print(f1[dset])

In [ ]:
files = [os.path.join(MOVES_FOLDER_PATH, f) for f in os.listdir(MOVES_FOLDER_PATH) if f.endswith('.json')]
for i,f in enumerate(files):
  from IPython.display import clear_output, display

  clear_output(wait=True)
  display(f'Processing file: {i}/{len(files)}')

  move_vars = loadMoves(f)

  trainHeld, testHeld,\
  trainActive, testActive,\
  trainPos, testPos,\
  trainRot, testRot,\
  trainGrids, testGrids,\
  trainY, testY = train_test_split(*move_vars, test_size=0.1, random_state=42)

  if i == 0:
    createDatasets(TRAIN_DATASET_FILE_PATH, trainHeld, trainActive, trainPos, trainRot, trainGrids.astype(np.int8), trainY)
    createDatasets(TEST_DATASET_FILE_PATH, testHeld, testActive, testPos, testRot, testGrids.astype(np.int8), testY)
  else:
    appendAllToDataset(TRAIN_DATASET_FILE_PATH, trainHeld, trainActive, trainPos, trainRot, trainGrids.astype(np.int8), trainY)
    appendAllToDataset(TEST_DATASET_FILE_PATH, testHeld, testActive, testPos, testRot, testGrids.astype(np.int8), testY)

printHDF5File(TRAIN_DATASET_FILE_PATH)
printHDF5File(TEST_DATASET_FILE_PATH)

'Processing file: 3836/8172'

KeyboardInterrupt: ignored

In [ ]:
printHDF5File(TRAIN_DATASET_FILE_PATH)
printHDF5File(TEST_DATASET_FILE_PATH)

<HDF5 dataset "X_active_block": shape (1066696,), type "<i8">
<HDF5 dataset "X_block_pos": shape (1066696, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (1066696,), type "<i8">
<HDF5 dataset "X_grid": shape (1066696, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (1066696,), type "<i8">
<HDF5 dataset "Y": shape (1066696,), type "<i8">
<HDF5 dataset "X_active_block": shape (268543,), type "<i8">
<HDF5 dataset "X_block_pos": shape (268543, 2), type "<i8">
<HDF5 dataset "X_block_rot": shape (268543,), type "<i8">
<HDF5 dataset "X_grid": shape (268543, 20, 10), type "|i1">
<HDF5 dataset "X_held_block": shape (268543,), type "<i8">
<HDF5 dataset "Y": shape (268466,), type "<i8">
